# Procesamiento de Texto con MoE - Análisis Avanzado

Este notebook demuestra las capacidades avanzadas del MoE para procesamiento de texto, incluyendo análisis detallado de tokens y visualización de métricas.

In [ ]:
import sys
sys.path.append('..')

from text_processing.text_moe import TextMoE
from utils.dashboard import ComplexityDashboard
import numpy as np
import matplotlib.pyplot as plt

## 1. Configuración y Análisis Inicial

Configuramos el TextMoE y analizamos su comportamiento con diferentes tipos de texto.

In [ ]:
text_moe = TextMoE(num_experts=3)
dashboard = ComplexityDashboard()

# Conjunto diverso de textos para análisis
textos = [
    "The quick brown fox jumps",  # Mezcla de longitudes
    "AI ML NLP deep learning",    # Términos técnicos
    "a an the but or and",        # Palabras cortas
    "extraordinary sophisticated" # Palabras largas
]

# Procesar y analizar cada texto
resultados = []
for texto in textos:
    print(f"\nProcesando: {texto}")
    resultado = text_moe.process(texto)
    print(resultado)
    
    # Recolectar métricas para visualización
    tokens = texto.split()
    for token in tokens:
        complejidad = text_moe._compute_complexity(token)
        resultados.append({
            'token': token,
            'complejidad': complejidad,
            'longitud': len(token)
        })

## 2. Visualización de Métricas

Analizamos la distribución de complejidad y la relación con la longitud de los tokens.

In [ ]:
def visualizar_metricas(resultados):
    complejidades = [r['complejidad'] for r in resultados]
    longitudes = [r['longitud'] for r in resultados]
    tokens = [r['token'] for r in resultados]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Distribución de complejidad
    ax1.hist(complejidades, bins=15, color='skyblue', alpha=0.7)
    ax1.set_title('Distribución de Complejidad')
    ax1.set_xlabel('Complejidad')
    ax1.set_ylabel('Frecuencia')
    
    # Relación longitud vs complejidad
    scatter = ax2.scatter(longitudes, complejidades, 
                         c=complejidades, cmap='viridis', 
                         alpha=0.6)
    ax2.set_title('Longitud vs Complejidad')
    ax2.set_xlabel('Longitud del Token')
    ax2.set_ylabel('Complejidad')
    plt.colorbar(scatter, ax=ax2, label='Complejidad')
    
    # Añadir etiquetas para algunos puntos interesantes
    for i, token in enumerate(tokens):
        if resultados[i]['complejidad'] > np.mean(complejidades) + np.std(complejidades):
            ax2.annotate(token, 
                        (longitudes[i], complejidades[i]),
                        xytext=(5, 5), textcoords='offset points')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar estadísticas
    print("\nEstadísticas de complejidad:")
    print(f"Media: {np.mean(complejidades):.2f}")
    print(f"Desviación estándar: {np.std(complejidades):.2f}")
    print(f"Máxima: {np.max(complejidades):.2f} ({tokens[np.argmax(complejidades)]})")
    print(f"Mínima: {np.min(complejidades):.2f} ({tokens[np.argmin(complejidades)]})")

visualizar_metricas(resultados)

## 3. Análisis de Patrones

Analizamos los patrones de asignación de expertos en un texto más largo.

In [ ]:
texto_largo = """
Natural Language Processing (NLP) combines linguistics and artificial intelligence 
to enable computers to process, understand, and generate human language effectively.
"""

def analizar_asignaciones(texto):
    tokens = texto.split()
    asignaciones = []
    complejidades = []
    
    for token in tokens:
        complejidad = text_moe._compute_complexity(token)
        expert_weights = text_moe._get_expert_weights(token)
        experto = np.argmax(expert_weights)
        
        asignaciones.append(experto)
        complejidades.append(complejidad)
        
        # Actualizar dashboard
        dashboard.update_metrics('text_complexity', complejidad)
        dashboard.update_metrics('expert_assignment', experto)
    
    return tokens, asignaciones, complejidades

tokens, asignaciones, complejidades = analizar_asignaciones(texto_largo)

# Visualizar distribución de expertos
expert_counts = np.bincount(asignaciones)
plt.figure(figsize=(8, 4))
plt.bar(range(len(expert_counts)), expert_counts)
plt.title('Distribución de Asignaciones por Experto')
plt.xlabel('Experto ID')
plt.ylabel('Número de Tokens')
plt.show()

# Mostrar asignaciones detalladas
print("\nAsignaciones detalladas:")
for token, experto, comp in zip(tokens, asignaciones, complejidades):
    print(f"Token: {token:15} | Experto: {experto} | Complejidad: {comp:.2f}")

## 4. Monitoreo en Tiempo Real

Simulamos procesamiento continuo y monitoreamos en tiempo real usando el dashboard.

In [ ]:
import time
import random
import string

def generar_token_aleatorio(min_len=1, max_len=15):
    length = random.randint(min_len, max_len)
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

def simular_procesamiento(num_tokens=50):
    for _ in range(num_tokens):
        token = generar_token_aleatorio()
        start_time = time.perf_counter()
        
        # Procesar token
        complejidad = text_moe._compute_complexity(token)
        expert_weights = text_moe._get_expert_weights(token)
        experto = np.argmax(expert_weights)
        
        # Calcular tiempo de procesamiento
        tiempo = time.perf_counter() - start_time
        
        # Actualizar dashboard
        dashboard.update_metrics('text_complexity', complejidad)
        dashboard.update_metrics('expert_assignment', experto)
        dashboard.update_metrics('processing_time', tiempo)
        
        time.sleep(0.1)  # Pausa para visualización

# Iniciar simulación y mostrar dashboard
print("Iniciando simulación de procesamiento...")
simular_procesamiento()